# 特征工程 #

## 一、从特征到特征工程 ##
&emsp;表格数据是根据由变量或属性（列）组成的变量或实例（行）来描述的。 通常我们所说的特征实际上是列中的属性，或者说，属性可以构成特征。在实际问题中，特征是指对我们的问题有意义的那些属性值。 是否对问题“有意义”可以用来来区分属性和特征。  如果某个属性对这个问题没有影响，那我们不应该把它认为是针对这个问题的一个特征。在计算机视觉中，图像是我们可以看到的，但其中的特征可能是图像中的某一条线。 在自然语言处理中，文档或推文可以是我们观察到的样本，而其中的短语或字数可以是特征。 在语音识别中，语音是我们观察到的数据样本，但是特征可以是单个词或音素。
&emsp;特征工程是将原始数据转化为特征，更好表示预测模型处理的实际问题，提升对于未知数据的准确性。它是用目标问题所在的特定领域知识或者自动化的方法来生成、提取、删减或者组合变化得到特征。
## 二、数据预处理(Pre-Processing) ##
### 2.1 规范数值变量 ###
&emsp;不同的特征可能会有不同的数值尺度和单位，举一个我最近碰到的情况为例，在文本分类中，我们常常用tf-idf来判断文章中出现的词语的重要性，并作为分类器的输入特征，每个单词的tf-idf值越大代表该单词的重要性越高，其值一般情况下都很小，仅采用tf-idf和逻辑回归模型进行分类的时候损失值比较低，但是当我加上内容长度、内容中各个不良词语出现个数的时候，损失值有了明显的飙升。我意识到这是因为新加入的特征和与tf-idf值的尺度差别太大，例如文本长度可能在1至数万之间。在这种情况下，标准化就很有必要了，常用的标准化方法有除以最大值、（原数据-极小值）/（极大值-极小值），新数据=1/（1+e^(-原数据)） 等等。 
&emsp; 需要注意的是，不是所有的特征都需要标准化，像经纬度这种特征如果经过标准化处理可能会导致有用信息的丢失，另外是否需要进行标准化也需要结合使用的机器学习模型来考虑，例如SVM,LR，K-means这些模型标准化能够提升性能，而XGBoost这种树提升模型则不需要进行数值型特征的标准化。

### 2.2 数值型特征向类别特征的转化 ###
&emsp; 有一些连续的数值型特征离散化处理后可以获得更好的效果，比如人的BMI值可以分成过轻、正常、超重、肥胖四个类别。通常可以按照分位点进行类别化处，为不同分段的特征采取不同的分类器，可能可以构建更具体、更准确、更具有针对性的模型。另外，分类型变量可能会有多个级别，比如邮政编码、商品编号中包含了很多层级，处理这种多层级的类别特征可能导致分类器的鲁棒性降低，因此在处理的时候要注意减少类别特征中的层级。在《Subscribe to Data Informed Enhance Machine Learning with Standardizing, Binning, Reducing》（http://data-informed.com/enhance-machine-learning-with-standardizing-binning-reducing/ ） 中，作者举了一个具体的例子，用户的位置总共有20个城市，但是95%的用户来自6个城市，那么我们可以把剩下的5%归纳到一个“other”中而不是一一列举，此外还可以引入“州”来替代用户来自的具体城市，这样减少了特征的维度，可以起到方便可视化、避免过拟合的效果。
&emsp 在处理类别特征的时候，还有一些其他技巧，比如引入哑变量。考虑这样一个情况，用户使用的操作系统有IOS,Android，Windows,如果我们用一个变量$x ∈ {0,1,2}$ 来表示这个特征的话，x=0和x=2尽管在数值上相差2，但是实际上并没有这种数值联系，仅仅代表这两个用户分别使用了IOS和Windows而已，但模型在处理时无法解释这种情况，因此，我们可以用两个变量$x_1∈ {0,1}$和$x_2∈ {0,1}$来表示用户使用的操作系统这一特征，n个类别的特征可以用n-1个哑变量来表示，这样增强了特征的解释性。
### 2.3 将非正态分布转化成正态分布 ###
&emsp; 数据正态化，目的是稳定方差，直线化，使数据分布正态或者接近正态。如果y = f(x) 是x的线性函数，不影响分析；但是如果是非线性函数，y和x的表现就完全不同，包括分布，方差和数据间关系也会不同。
&emsp; 比较常见的数据正态化方法包括取对数、取平方根、和取倒数。Box-Cox变换是一种很有效的数据正态化方法，变换之后，可以一定程度上减小不可观测的误差和预测变量的相关性。
&emsp Box-Cox的公式如下：
$$y(\lambda)=\begin{cases} 
		\dfrac{y_i^\lambda -1}{\lambda}, & \lambda \neq 0\\ 
		ln(y_i), & \lambda =0\
	\end{cases}$$
&emsp;其中$y_i$为样本点的值，y为变换后的值，我们只要确定合适的$\lambda$值就可以应用Box-Cox变换进行数据的正态化处理。那么我们如何来确定一个合适的λ值呢？ 我们将转化后的数据当做新的样本集合，计算特征值的平均值和方差，以此可以建立一个正态分布函数（我们假设转换后的样本集合符合正态分布），这样一来我们就可以计算每个样本点出现的概率了，通过最大似然法则，找到最合适的λ值。
### 2.4 缺失值填充 ###
&emsp; 除了比较朴素的按照平均值、0值等填充方法，用模型进行缺失值的预测或许会有更好的效果。例如数据集中有部分样本确实了“性别”属性值，我们可以利用决策树，将其他的部分特征作为输入来预测该属性值。如果属性值是连续的数值型，还可以通过逻辑回归进行填充。
## 三、特征提取(Feature Extraction) ##
&emsp;一些观测数据如果直接建模，其原始状态的数据太多。像图像、音频和文本数据，如果将其看做是表格数据，那么其中包含了数以千计的属性。特征提取是自动地对原始观测降维，使其特征集合小到可以进行建模的过程。
&emsp; 对于表格式数据，可以使用主元素分析(Principal Component Analysis)、聚类等映射方法；对于图像数据，可以进行线(line)或边缘(edge)的提取；根据相应的领域，图像、视频和音频数据可以有很多数字信号处理的方法对其进行处理。
&emsp 下面介绍一些关于特征提取的方法:
### 3.1 PCA ###
![](pca.png)
&emsp; PCA的是一种无监督的特征降维方法。PCA直接通过一个线性变换，将原始空间中的样本投影到新的低维空间中。简单来理解这一过程便是：PCA采用一组新的基来表示样本点，其中每一个基向量都是原来基向量的线性组合，通过使用尽可能少的新基向量来表出样本，从而达到降维的目的。
&emsp; PCA的步骤包括：（1） 求出每个特征的平均值,每个样例的该特征值均减去特征均值；（2）计算特征协方差矩阵，这里我们以每个样本三个特征的情况为例，$x_i=(a,b,c)$,$x_i$为第i个样本，a,b,c是三个不同的特征，则协方差矩阵为：$$
 \left[
 \begin{matrix}
   Cov(a,a) & Cov(a,b) & Cov(a,c) \\
   Cov(b,a) & Cov(b,b) & Cov(c,c) \\
   Cov(c,a) & Cov(c,b) & Cov(c,c)
  \end{matrix}
  \right] \tag{3}
$$
&emsp;其中$Cov(X，Y)=E[(X-E(X))(Y-E(Y))]$ 代表两个变量间的协方差。完成这一步骤后可以进行（3）求出协方差矩阵的特征值和特征向量，（4）按照协方差矩阵特征值从大到小排序，选择出前k个特征值和对应的特征向量。（5）最后，将完成第一步后得到的新数据点组成数据矩阵D（n,m）,与特征向量组成的矩阵F(m,k)相乘，就可以得到新的数据矩阵NewD（n,k）,其中n表示样本数量，m是原本的特征维度，k是降维后的特征维度。
### 3.2 MDS ###
&emsp;Multidimensional scaling，简称MDS，中文翻译成多维尺度分析。其原理是利用成对样本间的相似性，去构建合适的低维空间，使得样本在此空间的距离和在高维空间中的样本间的相似性尽可能的保持一致。MDS建立一个原始高维输入空间样本到低维特征空间样本的一一映射，建立的原则是在输入空间距离近的样本在低维特征空间的距离也要近。
&emsp; 假设我们有$n * d$的样本集$$
 X=\left[
 \begin{matrix}
   x_11 & x_12 &…… &x_1n \\
   x_21 & x_22 &…… &x_2n  \\
   …… & …… & …… &…… \\
   x_n1 & x_n2 &…… &x_nn
  \end{matrix}
  \right] \tag{4}
$$
&emsp; 之后我们计算出每两个样本点的欧氏距离构成距离矩阵$$
 d(X)=\left[
 \begin{matrix}
   d_11 & d_12 &…… &d_1n \\
   d_21 & d_22 &…… &d_2n  \\
   …… & …… & …… &…… \\
   d_n1 & d_n2 &…… &d_nn
  \end{matrix}
  \right] \tag{4}
$$
&emsp;接下来，我们需要把这n个数据点映射到更低维的向量空间Y中，在这个过程中，我们要使在原始空间中距离相近的点，在新的空间中仍然保持相近，损失函数为：$	\sigma(x)= \sum\limits_{i=1}^{n}{\sum\limits_{i=1}^{n}{(d_ij- \rVert \vec{y_i}-\vec{y_j}\rVert)^2}} $，我们可以采用基本的最优化理论来进行损失函数的优化：  

&emsp;  (1) 选择一个初始值 y=y0，找到一个函数,使得 $g_y(x)$,其中$g_y(x)≥f(x)$,假设 $x^*$ 是 gy(x) 的最小值，则根据$g_y(x)≥f(x)$，有 $f(x^*)≤g_y(x^*)，又因为 $g_y(x^*)$ 是函数 $g_y(x)$ 的最小值，则有 $g_y(x^2)≤$g_(y)$。所以，我们可以得到如下的一个不等式链： 
$f(x^*)≤g_y(x^*)≤g_y(y*)=f(y)$，找到满足（1）的$g_y(x)$后，继续往下执行。

&emsp;  (2) 第 i+1 步，找一个简单的，更容易求解最优值的函数 $g_y(x)$ 来代替求解 f(x),求解函数  $g_y(x)$ ，得到最小值  $y_{i+1}$。   

&emsp;  (3) 如果 $f(y_i)−f(y_{i+1})<ϵ$，则结束，f(x) 的最优值是 $f(y_{i+1})$；否则，另  $y=y_i+1$，转到步骤(2)  


解决这个优化问题，我们就可以得到一个满足我们要求的新的数据矩阵。我们把刚才提到过的目标函数进行展开：  
$$	\sigma(x)= \sum\limits_{i=1}^{n}{\sum\limits_{i=1}^{n}{(d_ij- \rVert \vec{y_i}-\vec{y_j}\rVert)^2}}=\sum\limits_{i=1}^{n}{\sum\limits_{i=1}^{n}{d_{ij}^2}+\sum\limits_{i=1}^{n}{ \rVert \vec{y_i}-\vec{y_j}\rVert ^2} -2\sum\limits_{i=1}^{n}{d_{ij}* \rVert \vec{y_i}-\vec{y_j}\rVert} }$$
&emsp; 观察上式，由于$d_{ij}$是一个常数，我们实际上要优化的函数为$$\sum\limits_{i=1}^{n}{ \rVert \vec{y_i}-\vec{y_j}\rVert ^2} -2\sum\limits_{i=1}^{n}{d_{ij}* \rVert \vec{y_i}-\vec{y_j}\rVert} $$ 可以结合柯西-施瓦茨不等式和刚才提到的最优化理论来解决这个问题，这里暂时没有给出具体的推导。

### 3.3 SVD###
&emsp; 在3.1中介绍的PCA方法里通过求协方差矩阵得到了一个方阵进而求出特征向量和特征值进行映射降维。 如果我们的数据矩阵恰巧是方阵，也可以直接进行特征向量分解进行降维，但现实中，恰好有n个数据，每个数据恰好是n维的情况少之又少。这种时候就可以借助奇异值分解的方法进行降维。$$
A=UΣV^T$$
&emsp; 假设A是一个N * M的矩阵，那么得到的U是一个N * N的方阵（里面的向量是正交的，U里面的向量称为左奇异向量），Σ是一个N * M的矩阵（除了对角线的元素都是0，对角线上的元素称为奇异值），V’(V的转置)是一个N * N的矩阵，里面的向量也是正交的，V里面的向量称为右奇异向量），从图片来反映几个相乘的矩阵的大小可得下面的图片:  
![](svd.png)
&emsp; 那么现在我们就需要确定左奇异向量和右奇异向量。$A*A^T$会得到一个方阵，我们对这个方阵进行特征分解：$$
AA^T\lambda _i = \lambda_i v_i$$ 由此可以求出特征值和特征向量，这个步骤中求出的特征向量就是右奇异向量，特征值为右奇异值，此外我们可以得到$$ \sigma_i=\sqrt(\lambda_i)$$ $$u_i=Av_i/\sigma i$$ 这就是左奇异值和左奇异向量。
&emsp; 奇异值σ跟特征值类似，在矩阵Σ中也是从大到小排列，而且σ的减少特别的快，在很多情况下，前10%甚至1%的奇异值的和就占了全部的奇异值之和的99%以上了。也就是说，我们也可以用前r大的奇异值来近似描述矩阵，这里定义一下部分奇异值分解：
$$ A\approx U_{m * r}Σ_{r*r} V^T_{n*n} $$,其中r是一个远远小于n的数，这样就起到了降维的效果。

## 四、特征选择(Feature Selection) ##
$emsp; 尽管有些特征对我们的问题是有意义的，但是不同特征的重要性不同，有些特征之间可能存在了信息冗余，过多的特征也可能会导致过拟合的情况发生，因此选出一些对问题影响比较大的特征就尤为重要了。
&emsp;  特征选择是从原始的特征集中选择出一个子集，从而在不降低甚至提高模型性能的情况下，减少输入数据的规模。 
&emsp;  按照是否需要标签，特征选择可以分为有监督特征选择和无监督特征选择。根据特征选择的形式又可以将特征选择方法分为3种：Filter：过滤法，按照发散性或者相关性对各个特征进行评分，设定阈值或者待选择阈值的个数，选择特征。Wrapper：包装法，根据目标函数（通常是预测效果评分），每次选择若干特征，或者排除若干特征。Embedded：嵌入法，先使用某些机器学习的算法和模型进行训练，得到各个特征的权值系数，根据系数从大到小选择特征。类似于Filter方法，但是是通过训练来确定特征的优劣。
### 4.1过滤法###
&emsp; 过滤类方法采用代理指标，而不根据特征子集的错误率计分。所选的指标算得快，但仍然能估算出特征集好不好用。常用指标包括互信息、逐点互信息、皮尔逊积矩相关系数、每种分类/特征的组合的帧间/帧内类距离或显著性测试评分。  过滤类方法计算量一般比包装类小，但这类方法找到的特征子集不能为特定类型的预测模型调校。由于缺少调校，过滤类方法所选取的特征集会比包装类选取的特征集更为通用，往往会导致比包装类的预测性能更为低下。不过，由于特征集不包含对预测模型的假设，更有利于暴露特征之间的关系。许多过滤类方法提供特征排名，而非显式提供特征子集。要从特征列表的哪个点切掉特征，得靠交叉验证来决定。过滤类方法也常常用于包装方法的预处理步骤，以便在问题太复杂时依然可以用包装方法。
#### 4.1.1 皮尔逊相关系数法 ####
&emsp;皮尔逊相关系数法可以用来衡量两个变量之间的线性相关性，皮尔逊系数取值范围为[-1,+1],当系数值为0的时候，代表两个变量之间是线性无关的，而取值为+1或者-1的时候，代表两个变量之间有y=kx（k≠0）的线性关系。对于n个样本的两个不同的变量x,y，皮尔逊系数的计算公式为：
$$ d(x,y)= \sqrt { ((x_1-y_1)^2+(x_2-y_2)^2 +……+(x_n-y_n)^2) } $$
&emsp; 我们可以计算每个特征x和标签y之间的皮尔逊相关系数，以此决定特征的重要程度，进行特征选择。但有一点需要注意，尽管某个特征x和标签y之间的皮尔逊相关系数很小，也不代表二者的关联程度小，只能说他们之间的线性关联程度小，而非线性关系无法用皮尔逊系数进行衡量。
#### 4.1.2 基于互信息的特征选择 ####
&emsp;由于皮尔逊相关系数只能反映变量之间的线性相关性，因此互信息更能衡量特征与标签之间的关联程度。互信息是用来评价一个事件的出现对于另一个事件的出现所贡献的信息量，经典的互信息计算公式为：
$$I(X,Y)=\sum_X \sum_Y {p(x,y) log \frac{p(x,y)}{p(x)p(y)}}$$  


&emsp; 很明显这种方法不适合连续型变量的特征选择，因此我们通常采用最大信息系数来计算，我们把两个随机变量化成散点图，然后不断的用小方格子去分割。然后计算每个方格子里面的落入概率。在某种意义上，就可以估计出联合概率密度分布了。当然，只有在数据量是无穷的情况下我们才可以认为是真的就相等了。所以，导致随后是数据量越大，MIC越好。
$$ MIC=Max_{|X||Y|<B} \frac{I[X;Y]}{log_2 min(|X|,|Y|)} $$

|X|就是X方向共被分成了多少段的意思。Y方向也一样。前面还有一个限制条件，就是|X||Y|<B，也就是说，所有的方格格总数不能大于B。作者说，B取数据总量的0.6或者0.55次方。B的取值没有详细的解释，可能是一个经验性的结论。

### 4.2包装法###
&emsp;包装类方法使用预测模型给特征子集打分。每个新子集都被用来训练一个模型，然后用验证数据集来测试。通过计算验证数据集上的错误次数（即模型的错误率）给特征子集评分。由于包装类方法为每个特征子集训练一个新模型，所以计算量很大。不过，这类方法往往能为特定类型的模型找到性能最好的特征集。

&emsp;常用的包装法包括了前向选择法，后向剔除法，迭代剔除法等。  

&emsp;前向选择法：这是一种基于循环的方法，开始时我们训练一个不包含任何特征的模型，而后的每一次循环我们都持续放入能最大限度提升模型的变量，直到任何变量都不能提升模型表现。

&emsp;后向剔除法：该方法先用所有特征建模，再逐步剔除最不显著的特征来提升模型表现。同样重复该方法直至模型表现收敛。

&emsp;迭代剔除法：这是一种搜索最优特征子集的贪心优化算法。它会反复地训练模型并剔除每次循环的最优或最劣特征。下一次循环，则使用剩余的特征建模直到所有特征都被剔除。之后，按照剔除的顺序给所有特征排序作为特征重要性的度量。

&emsp; 也有学者利用神经网络对传统的包装法进行改进，在“A new wrapper feature selection approach using neural network”这篇论文中，作者将初始的特征集根据特征之间的相似程度均等地分为两个子集S和D，建立一个三层的神经网络结构，并分别从S和D中选择一个与同特征子集中其他特征相似度最小的特征加入训练，通过该神经网络以及输入的特征子集进行训练，通过预测结果与实际标签的比较判定是否选择该特征，如果当前输入的特征不符合要求，则增加隐层单元数量并且选择新的特征。模型的结构图如下所示：  

![](CAFS.png)
### 4.3嵌入法###
&emsp;嵌入类方法包括了所有构建模型过程中用到的特征选择技术。这类方法的典范是构建线性模型的LASSO方法。该方法给回归系数加入了L1惩罚，导致其中的许多参数趋于零。任何回归系数不为零的特征都会被LASSO算法“选中”。LASSO的改良算法有Bolasso和FeaLect。Bolasso改进了样本的初始过程。FeaLect根据回归系数组合分析给所有特征打分。 另外一个流行的做法是递归特征消除（Recursive Feature Elimination）算法，通常用于支持向量机，通过反复构建同一个模型移除低权重的特征。这些方法的计算复杂度往往在过滤类和包装类之间。
![](embedding.png)
